In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from numpy.random import seed

In [2]:
from pandas.io.json import json_normalize 
data = pd.read_csv('/Users/Sandi/Desktop/Projects/Capstone_2/questionnaire.csv')
data.head()

,SEQN,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,...,WHD080U,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520
0,73557,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
1,73558,1.0,NaN,NaN,NaN,NaN,1.0,NaN,7.0,1.0,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
2,73559,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
3,73560,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0
4,73561,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,150.0,135.0,67.0,170.0,60.0,NaN,NaN,NaN


In [3]:
#https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Questionnaire&CycleBeginYear=2013
##The variable dictionary to understand the code at the top of each column.  

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10175 entries, 0 to 10174
Columns: 953 entries, SEQN to WHQ520
dtypes: float64(944), int64(7), object(2)
memory usage: 74.0+ MB


Medical Conditions  MCQ084 
difficulties in thinking or remembering that can make a big difference in everyday activities.
The next question asks about difficulties in thinking or remembering that can make a big difference in everyday activities. This does not refer to occasionally forgetting your keys or the name of someone you recently met. This refers to things like confusion or memory loss that are happening more often or getting worse. We want to know how these difficulties impact {you/SP}. During the past 12 months, {have you/has she/has he} experienced confusion or memory loss that is happening more often or is getting worse?

In [5]:
data.MCQ084.value_counts()

2.0    1521
1.0     318
9.0       1
Name: MCQ084, dtype: int64

In [11]:
data.MCQ084.count()

1840

# https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/MCQ_H.htm#MCQ084
- 1521 is the count of No
- 318 is the count of Yes
- 1 is the count of don't know
- 7,930 are missing

In [6]:
data.MCQ084.tail(10)

10165    NaN
10166    2.0
10167    1.0
10168    NaN
10169    NaN
10170    NaN
10171    NaN
10172    NaN
10173    NaN
10174    NaN
Name: MCQ084, dtype: float64

In [7]:
#drop the NaN values.  How many rows re
memory_difficulty= data.MCQ084.dropna(axis = 0)
memory_difficulty.tail(10)



10127    2.0
10130    2.0
10135    2.0
10145    2.0
10151    2.0
10155    2.0
10160    2.0
10161    1.0
10166    2.0
10167    1.0
Name: MCQ084, dtype: float64

In [9]:
memory_difficulty.count()

1840

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/PAQ_H.htm
-Past wk # days cardiovascular exercise  Physical Activity PAQ677
the first column is the number od days, the second column is the count
8805 are missing

Physical Activity PAQ677 On how many of the past 7 days did {you/SP} exercise or participate in physical activity for at least 20 minutes that made {you/him/her} sweat and breathe hard, such as basketball, soccer, running, swimming laps, fast bicycling, fast dancing, or similar activities?

In [12]:
data.PAQ677.value_counts()

5.0     154
0.0     111
3.0     110
2.0      81
4.0      65
1.0      62
7.0      52
6.0      43
99.0      1
Name: PAQ677, dtype: int64

Physical Activity PAQ678 On how many of the past 7 days did {you/SP} do exercises to strengthen or tone {your/his/her} muscles, such as push-ups, sit-ups, or weight lifting?

In [13]:
data.PAQ678.value_counts()

0.0    207
2.0    109
3.0    106
1.0     96
5.0     77
4.0     39
7.0     32
6.0     13
Name: PAQ678, dtype: int64

Physical Activity PAQ706 During the past 7 days, on how many days {were you/was SP} physically active for a total of at least 60 minutes per day? Add up all the time {you/he/she} spent in any kind of physical activity that increased {your/his/her} heart rate and made {you/him/her} breathe hard some of the time.

In [15]:
data.PAQ706.value_counts()

7.0     1799
5.0      314
3.0      194
4.0      183
2.0      159
0.0      131
6.0      111
1.0       90
99.0       8
Name: PAQ706, dtype: int64